# MFCCS
Obtener los mfccs para 5 clases c/u de 500 audios, para un total de 15.000 audios

In [97]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing

In [74]:
PATHSAMPLES = '../data/training_list_test_temporal.txt'
#PATHSAMPLES = '../data/training_list.txt'
NSAMPLES = 3
#SOUNDS =np.array(['yes','no','right','five','nine'])
SOUNDS =np.array(['off','five','left'])

In [75]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/audios_test/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    return matrixMajor

In [76]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [77]:
path_word_speakerId.shape

(9, 3)

In [78]:
path_word_speakerId

array([['data/audios_test/off/0a2b400e_nohash_0.wav', 'off', '0a2b400e'],
       ['data/audios_test/off/0a2b400e_nohash_1.wav', 'off', '0a2b400e'],
       ['data/audios_test/off/0a2b400e_nohash_2.wav', 'off', '0a2b400e'],
       ['data/audios_test/five/0a2b400e_nohash_0.wav', 'five',
        '0a2b400e'],
       ['data/audios_test/five/0a2b400e_nohash_1.wav', 'five',
        '0a2b400e'],
       ['data/audios_test/five/0a2b400e_nohash_2.wav', 'five',
        '0a2b400e'],
       ['data/audios_test/left/0a2b400e_nohash_0.wav', 'left',
        '0a2b400e'],
       ['data/audios_test/left/0a2b400e_nohash_1.wav', 'left',
        '0a2b400e'],
       ['data/audios_test/left/0a2b400e_nohash_2.wav', 'left',
        '0a2b400e']], dtype='<U43')

In [246]:
def build_features(path_word_speakerId, n_mfcc,NSAMPLES, PATHSAMPLES, SOUNDS):
    N_rows = NSAMPLES*len(SOUNDS)*n_mfcc
    N_colums = n_mfcc +3# número de columnas del mfcc + 3 columnas (la palabra y el speaker id, path al que pertenece)
    # el path es necesario ya que estamos teniendo un enfoque de multiples instancias.
    path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)
    matrix_features = np.empty((1,23)) # inicialmente

    #Encode etiqueta del audio,speaker y la palabra
    lePath = preprocessing.LabelEncoder()
    leSpeaker = preprocessing.LabelEncoder()
    leWord =  preprocessing.LabelEncoder()
    
    lePath.fit(path_word_speakerId[:,0])
    leSpeaker.fit(path_word_speakerId[:,2])
    leWord.fit(path_word_speakerId[:,1])
    
        
    for sound_info in path_word_speakerId:
        audio_path = '../'+sound_info[0]
        x , sr = librosa.load(audio_path)
        x_normalize=sk.preprocessing.minmax_scale(x, axis=0)       
        mfccs = librosa.feature.mfcc(x_normalize, sr=sr,n_mfcc=n_mfcc,hop_length=int(0.010*sr), n_fft=int(0.025*sr))
        n_windows = mfccs.shape[1]
        path = sound_info[0]
        speaker_id = sound_info[2]
        word=sound_info[1]        
        labelPath = lePath.transform([path])
        labelSpeaker = leSpeaker.transform([speaker_id])
        labelWord= leWord.transform([word])                
        labelPath = np.repeat(labelPath,n_windows)
        labelWord =  np.repeat(labelWord,n_windows)
        labelSpeaker= np.repeat(labelSpeaker,n_windows)        
        fila_P_W_S = np.array([labelPath,labelWord,labelSpeaker]).T        
        mfccFull =  np.hstack((mfccs.T,fila_P_W_S))        
        matrix_features = np.concatenate((matrix_features,mfccFull), axis=0)        
    return matrix_features[1:,:]

In [247]:
matrixFinal = build_features(path_word_speakerId, 20,NSAMPLES, PATHSAMPLES, SOUNDS)
print(matrixFinal.shape)

(909, 23)


In [248]:
matrixFinal.shape

(909, 23)

In [249]:
matrixFinal

array([[-657.64953613,   19.85280228,   19.83027458, ...,    6.        ,
           2.        ,    0.        ],
       [-657.64953613,   19.85275841,   19.83023071, ...,    6.        ,
           2.        ,    0.        ],
       [-657.64953613,   19.8527565 ,   19.8302269 , ...,    6.        ,
           2.        ,    0.        ],
       ...,
       [-660.11584473,   21.35703278,   20.38794899, ...,    5.        ,
           1.        ,    0.        ],
       [-659.33129883,   22.92096329,   22.72964859, ...,    5.        ,
           1.        ,    0.        ],
       [-659.78820801,   22.10509109,   21.59470749, ...,    5.        ,
           1.        ,    0.        ]])

In [ ]:
np.savetxt('data/audios_MFCC.csv',matrixFinal,delimiter=",")